In [15]:
from jp_doodle import doodle_files
vf_js = doodle_files.vendor_path("js/canvas_2d_vector_field.js")
from jp_doodle import dual_canvas
import jp_proxy_widget

In [16]:
w = jp_proxy_widget.JSProxyWidget()
w.load_js_files([vf_js])
dual_canvas.load_requirements(w)
w.js_init("""
debugger;
element.canvas_2d_vector_field.example(element);
""")

w.debugging_display()

VBox(children=(JSProxyWidget(status=u'Not yet rendered'), Text(value=u'Not yet rendered', description=u'status:'), Text(value=u'No error', description=u'error'), Output(layout=Layout(border=u'1px solid black'))))

In [3]:
w.element.reset_canvas()

element

In [17]:
import numpy as np
print np.linalg.norm((2,3))
x = np.random.random((2,3))

def unitize(vectors):
    (nr, nc) = vectors.shape
    norms = np.linalg.norm(vectors, axis=1)
    return vectors/norms.reshape((nr, 1))

(x, unitize(x))


3.60555127546


(array([[ 0.30242086,  0.57327401,  0.87441002],
        [ 0.8699484 ,  0.32549768,  0.29876122]]),
 array([[ 0.27784884,  0.52669488,  0.80336326],
        [ 0.89160212,  0.33359958,  0.30619763]]))

In [18]:
# random motions from 1000 random positions
npoints = 100
duration = 1
shape = (npoints, 2)
points = np.random.random(shape) * 2.0 - 1.0
max_movement = 0.2
movement = max_movement * (np.random.random(shape) * 2.0 - 1.0)
offsets = np.random.random((npoints,)) * duration
motions = []
for i in range(npoints):
    (x, y) = points[i]
    (dx, dy) = movement[i]
    motion = {
        "sx": x-dx, "sy": y-dy, "ex": x+dx, "ey": y+dy, "dt": offsets[i]
    }
    motions.append(motion)

In [19]:
def show_motions(motions, duration=1, radius=0.01):    
    #c2 = dual_canvas.DualCanvasWidget(width=800, height=800)
    c2 = dual_canvas.SnapshotCanvas("vector_field.png", width=800, height=800)
    c2.js_init("""
            var frame0 = element.frame_region(50, 50, 300, 300, -1, -1, 1, 1);
            element.frame0 = frame0;
            frame0.frame_rect({x:-1, y:-1, w:2, h:2, color:"#ddf"});
            frame0.lower_left_axes({min_x:-1, min_y:-1, max_x:1, max_y:1, max_tick_count:4});
            var frame = element.frame_region(50, 50, 300, 300, -1, -1, 1, 1);
            var f_options = {
                frame: frame,
                duration: duration,  // seconds
                initial_color: "red",
                final_color: "red",
                radius: radius,
                motions: motions,
                shape: "line",
            };
            element.fit(null, 50)
            element.canvas_2d_vector_field(f_options);
    """, motions=motions, duration=duration, radius=radius)
    #return c2.debugging_display()
    return c2

c2 = show_motions(motions, duration)
c2.display_all()

Tab(children=(VBox(children=(SnapshotCanvas(status=u'Not yet rendered'), JSProxyWidget(status=u'Not yet rendered'))), HTML(value=u'<img src="vector_field.png?id=snapshot_id_9_1539974981245" id="snapshot_id_9_1539974981245"/>\n <div id="snapshot_id_10_1539974981246">vector_field.png</div>')), _titles={u'1': 'Snapshot', u'0': 'Canvas'})

In [20]:
c2.element.reset_canvas()

element

In [21]:
#points

In [22]:
#motions

In [23]:
# attraction towards the center
xc = 0.3
yc = 0.4
npoints = 100
duration = 1
shape = (npoints, 2)
points = np.random.random(shape) * 2.0 - 1.0
max_movement = 0.2
shift = np.array([[xc, yc]]) - points
movement = max_movement * shift
offsets = np.random.random((npoints,)) * duration
motions = []
for i in range(npoints):
    (x, y) = points[i]
    (dx, dy) = movement[i]
    motion = {
        "sx": x-dx, "sy": y-dy, "ex": x+dx, "ey": y+dy, "dt": offsets[i]
    }
    motions.append(motion)
    
c3 = show_motions(motions, duration)
#c3

In [24]:
# motion around the center point
xc = 0.3
yc = 0.4
npoints = 100
duration = 1
shape = (npoints, 2)
points = np.random.random(shape) * 2.0 - 1.0
max_movement = 0.2
shift = np.array([[xc, yc]]) - points
shift2 = np.zeros(shape)
shift2[:,0] = shift[:,1]
shift2[:,1] = -shift[:,0]
movement = max_movement * shift2
offsets = np.random.random((npoints,)) * duration
motions = []
for i in range(npoints):
    (x, y) = points[i]
    (dx, dy) = movement[i]
    motion = {
        "sx": x-dx, "sy": y-dy, "ex": x+dx, "ey": y+dy, "dt": offsets[i]
    }
    motions.append(motion)
    
c4 = show_motions(motions, duration)
c4.js_init("""
debugger;
element.frame0.frame_circle({x: xc, y:yc, r:max_movement, color:"rgba(255,200,100,0.7)"})
""", xc=xc, yc=yc, max_movement=max_movement)
#c4

In [25]:
c4.element.reset_canvas()

element

In [26]:
def attraction(cx, cy, points, factor=1.0):
    shift = factor * (np.array([[cx, cy]]) - points)
    return unitize(shift)

def clockwise(cx, cy, points, factor=1.0):
    shift = attraction(cx, cy, points, factor=1.0)
    shift2 = np.zeros(shift.shape)
    shift2[:,0] = shift[:,1]
    shift2[:,1] = -shift[:,0]
    return shift2

npoints = 1000
duration = 1
shape = (npoints, 2)
points = np.random.random(shape) * 2.0 - 1.0
shift = 0.25 * (
    #+ attraction(0.5, 0.5, points)
    + 2* attraction(-0.5, -0.5, points)
    + clockwise(0.5, -0.5, points, -1)
    + 3*clockwise(-0.5, 0.5, points, -1)
)
#shift = unitize(shift)
max_movement = 0.2
movement = max_movement * shift
offsets = np.random.random((npoints,)) * duration
motions = []
for i in range(npoints):
    (x, y) = points[i]
    (dx, dy) = movement[i]
    motion = {
        "sx": x-dx, "sy": y-dy, "ex": x+dx, "ey": y+dy, "dt": offsets[i]
    }
    motions.append(motion)
    
c4 = show_motions(motions, duration)
c4

SnapshotCanvas(status=u'Not yet rendered')

In [14]:
c4.element.reset_canvas()

element

In [ ]:
# dump motions to file
filename = "motions.json"
f = open(filename, "w")
f.write("[\n")
for m in motions:
    f.write("{")
    for k in m:
        v = m[k]
        f.write("%s:%2.2f," % (k, v))
    f.write("},\n")
f.write("]\n")
f.close()

In [ ]:
print(open(filename).read(3200))

In [ ]:
motions[0]